# Configurações Iniciais

In [1]:
# Importando bibliotecas
!pip install xgboost
import numpy as np
from numpy import mean, std
import pandas as pd
import seaborn as sns
import tkinter
import plotly.express as px
from collections import Counter
from matplotlib import pyplot as plt
from matplotlib.lines import Line2D
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import OneHotEncoder, StandardScaler, OrdinalEncoder
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV
from sklearn.metrics import *
from sklearn.feature_selection import RFECV
import xgboost as xgb
from sklearn.pipeline import Pipeline
from sklearn.model_selection import RepeatedKFold
from prophet import Prophet
from sklearn.feature_selection import SelectFromModel


# pd.set_option('display.max_rows', None)
# Retirar os limitadores de coluna
pd.set_option('display.max_columns', None)

# Unindo as bases de dados e removendo as não utilizadas

df_2020 = pd.read_csv(r"C:\Users\PC\Desktop\BASES MOBLY-20230731T191159Z-001 (1)\BASES MOBLY\base_inteli 2020_2021.csv", ';')
df_2022 = pd.read_csv(r"C:\Users\PC\Desktop\BASES MOBLY-20230731T191159Z-001 (1)\BASES MOBLY\base_inteli_2022_2023.csv", ';')
df_dolar_2020 = pd.read_csv(r"C:\Users\PC\Downloads\USDBRL(2020).csv")
df_dolar_2021 = pd.read_csv(r"C:\Users\PC\Downloads\USDBRL(2021).csv")
df_dolar_2022 = pd.read_csv(r"C:\Users\PC\Downloads\USDBRL(2022).csv")
df_dolar_2023 = pd.read_csv(r"C:\Users\PC\Downloads\USDBRL(2023).csv")
df_all = pd.concat([df_2020, df_2022])
df_dolar = pd.concat([df_dolar_2020, df_dolar_2021, df_dolar_2022, df_dolar_2023])

C:\Users\PC\AppData\Local\Temp\ipykernel_20780\135771212.py:33: FutureWarning: In a future version of pandas all arguments of read_csv except for the argument 'filepath_or_buffer' will be keyword-only.
  df_2020 = pd.read_csv(r"C:\Users\PC\Desktop\BASES MOBLY-20230731T191159Z-001 (1)\BASES MOBLY\base_inteli 2020_2021.csv", ';')
C:\Users\PC\AppData\Local\Temp\ipykernel_20780\135771212.py:34: FutureWarning: In a future version of pandas all arguments of read_csv except for the argument 'filepath_or_buffer' will be keyword-only.
  df_2022 = pd.read_csv(r"C:\Users\PC\Desktop\BASES MOBLY-20230731T191159Z-001 (1)\BASES MOBLY\base_inteli_2022_2023.csv", ';')


***Adicionando cotação do dolar no dataframe***

In [3]:
df_dolar.rename(columns={"Date": "date"}, inplace=True)
df_dolar.rename(columns={"Close": "dollar_quotation"}, inplace=True)
df_dolar = df_dolar.drop(['Open', 'High', 'Low', 'Adj Close', 'Volume'], axis=1)
df_dolar['dollar_quotation'] = df_dolar['dollar_quotation'].apply(lambda x: round(x, 4))
df_all = pd.merge(df_all, df_dolar, how='left')
df_all['dollar_quotation'].fillna(method='ffill', inplace=True)

In [4]:
df_dolar.head(50)

,date,dollar_quotation
0,2020-01-01,4.0168
1,2020-01-02,4.0163
2,2020-01-03,4.0234
3,2020-01-06,4.0570
4,2020-01-07,4.0604
5,2020-01-08,4.0662
6,2020-01-09,4.0628
7,2020-01-10,4.0921
8,2020-01-13,4.0705
9,2020-01-14,4.1458


In [5]:
# Convert the 'Date' column to datetime
df_dolar['date'] = pd.to_datetime(df_dolar['date'])

# Set 'Date' as the index
df_dolar.set_index('date', inplace=True)

# Resample to weekly frequency and calculate the mean
df_dolar_semanal = df_dolar['dollar_quotation'].resample('W').mean()
df_dolar_semanal

date
2020-01-05    4.018833
2020-01-12    4.067700
2020-01-19    4.141080
2020-01-26    4.182280
2020-02-02    4.206020
2020-02-09    4.259500
2020-02-16    4.334040
2020-02-23    4.346920
2020-03-01    4.419420
2020-03-08    4.523300
2020-03-15    4.711760
2020-03-22    5.011900
2020-03-29    5.068320
2020-04-05    5.200860
2020-04-12    5.216960
2020-04-19    5.186780
2020-04-26    5.370620
2020-05-03    5.508680
2020-05-10    5.627920
2020-05-17    5.774940
2020-05-24    5.714460
2020-05-31    5.398480
2020-06-07    5.215880
2020-06-14    4.925340
2020-06-21    5.209940
2020-06-28    5.283800
2020-07-05    5.403960
2020-07-12    5.344780
2020-07-19    5.358560
2020-07-26    5.242300
2020-08-02    5.169100
2020-08-09    5.290460
2020-08-16    5.420020
2020-08-23    5.501360
2020-08-30    5.582500
2020-09-06    5.381360
2020-09-13    5.317780
2020-09-20    5.266520
2020-09-27    5.474620
2020-10-04    5.620440
2020-10-11    5.611740
2020-10-18    5.565780
2020-10-25    5.610740
2020-1

In [6]:
df_all = pd.merge(df_all, df_dolar_semanal, how='left')

# Exploração dos dados


###Identificação das colunas

A identificação das colunas desempenha um papel importante no processo de exploração. Através dela, podemos conhecer mais sobre os tipos de dados presente em cada coluna e direcionar o tratamento adequado que iremos utilizar para cada caso.

In [ ]:
df_all.info()

Através do método `info()`, foi possível classificar nossas características em numéricas ou categóricas:

**Colunas numéricas**

---

* unit_price
* sku_height
* sku_width
* sku_length
* sku_weight
* winning_price
* revenue
* items_sold
* avg_website_visits_last_week
* supplier_delivery_time
* stock_qty
* revenue_bundle
* items_sold_bundle
<br>
<br>

**Colunas categóricas**

---



* date
* weekday_name
* sku
* mobly_item
* shipment_type
* anchor_category
* product_department
* product_category
* origin_country
* process_costing
* sku_color
* price_status
* flag_bundle

###Estatística descritiva das colunas



A estatística descritiva é uma parte da estatística que envolve a coleta, organização, resumo e interpretação de dados para descrever características importantes de um conjunto de informações. Ela se concentra em transformar dados brutos em informações compreensíveis e significativas, permitindo uma compreensão mais clara das tendências, padrões e distribuições presentes nos dados.

Para o caso do nosso projeto, a estatística descritiva desempenha um papel importante para a ideação de um modelo preditivo de regressão. Antes de construir esse modelo, é fundamental entender a distribuição dos dados históricos de vendas. Então, a estatística descritiva, como a média, mediana, desvio padrão e percentis, ajuda a identificar não apenas as tendências, mas também os outliers nos dados de vendas passados.

O método nativo `describe()` da biblioteca Pandas foi utilizado para fazer a análise exploratória dos dados, já que fornece estatísticas para cada uma das colunas do dataframe.

In [ ]:
df_all.describe()

Com o banco de dados original apresentando algumas informações faltantes em certas linhas, as estatísticas descritivas puderam ser utilizadas para imputar os dados ausentes. Isso pôde ser feito por meio da utilização da mediana, por exemplo, atribuindo o valor dessa tendência central nos campos em branco.

# Pré-processamento


###Limpeza de dados


**Tratamento de missings e remoção das colunas**

---
Removemos as colunas que possuíam somente um valor exclusivo, aquelas que apresentavam mais de 50% dos seus valores vazios ou sem registro e as que continham valores irrelevantes para o nosso modelo.

As colunas possuíam poucos missing values, apenas presentes em algumas dimensões de sku’s, então optamos por substituir esses valores por 0, visto que a inexistência de informações sobre suas dimensões não alterava em sua venda.

In [ ]:
# Função para encontrar o índice da primeira venda ou visita
def find_first_occurrence(df_all):
    condition = (df_all['items_sold'] > 0) | (df_all['avg_website_visits_last_week'] > 0)
    if condition.any():
        return condition.idxmax()
    else:
        return None

# Aplica a função para cada grupo de Produto e cria a máscara
mask = df_all.groupby('sku').apply(find_first_occurrence).reset_index(name='first_occurrence_index')
df_all = df_all.merge(mask, on='sku', how='left')
keep_rows = df_all.index >= df_all['first_occurrence_index']

# Filtrar o dataframe usando a máscara
df_all = df_all[keep_rows].drop(columns=['first_occurrence_index'])

display(df_all.sort_values('date'))

In [ ]:
df_fixed = df_all.drop(["mobly_item",'price_status','winning_price', 'revenue','revenue_bundle'], axis ='columns').fillna(0).reset_index(drop = True)
display(df_fixed)

In [ ]:
df_fixed = df_fixed[df_fixed['items_sold'] != -1].reset_index()

### Adequação de variáveis numéricas e categóricas


**Separação das variáveis**

---
*(Revisar e corrigir)*<BR>
Todas as colunas identificadas como categóricas em nossa database foram percorridas, contando o número de valores únicos presentes. Com base nessa contagem, foi possível compreender a variedade de categorias por atributo, separando as colunas pelo método que seriam codificadas: One-Hot Encoder ou Label Encoder.

In [ ]:
columns = list(df_fixed.columns)

categoric_columns = []
numeric_columns = []

for i in columns:
    if len(df_fixed[i].unique()) > 7:
        numeric_columns.append(i)
    else:
        categoric_columns.append(i)



print('Label Encoding: ',numeric_columns)
print('One-Hot Encoding: ',categoric_columns)

**One-Hot Encoder**


---

Empregamos o One-Hot Encoder em colunas categóricas com até 7 elementos. Essa escolha foi feita para converter essas colunas em um formato binário sem sobrecarregar o sistema. O processo desse codificador gera uma coluna nova para representar cada categoria, na qual o valor é atribuído como "1" ou "0" para indicar se está presente ou ausente, respectivamente.





In [ ]:
# Selecionando as colunas para sofrerem codificação
columns_to_encode = ['shipment_type', 'product_department', 'origin_country', 'process_costing', 'flag_bundle']
data_to_encode = df_fixed[columns_to_encode]

# Transformando as colunas para um novo DataFrame
encoder = OneHotEncoder(sparse=False)
encoded = encoder.fit_transform(data_to_encode)
df_oneHot = pd.DataFrame(encoded, columns=encoder.get_feature_names_out(columns_to_encode))

# Substituir as colunas originais pelas colunas do df_oneHot
df_fixed_encoded = df_fixed.drop(columns=columns_to_encode)  # Remover as colunas originais
df_fixed_encoded = pd.concat([df_fixed_encoded, df_oneHot], axis=1)  # Concatenar as colunas codificadas

display(df_fixed_encoded)

**Label Encoder**


---

Utilizamos o Label Encoder em colunas categóricas com 7 ou mais elementos. Nesse processo, cada item da categoria era atribuído a um número inteiro distinto, não expandindo o número de colunas.




In [ ]:
# Agrupa os items vendidos de acordo com cada item de uma coluna
color_sums = df_fixed_encoded.groupby(['sku_color'])['items_sold'].sum().sort_values(ascending=True)
sku_sums = df_fixed_encoded.groupby(['sku'])['items_sold'].sum().sort_values(ascending=True)
anchor_sums = df_fixed_encoded.groupby(['anchor_category'])['items_sold'].sum().sort_values(ascending=True)
product_sums = df_fixed_encoded.groupby(['product_category'])['items_sold'].sum().sort_values(ascending=True)

# Cria um dicionário mapeando as cores de forma ordinal (Maior para menor)
color_ordinal_mapping = {color: i for i, color in enumerate(color_sums.index)}
sku_ordinal_mapping = {sku: i for i, sku in enumerate(sku_sums.index)}
anchor_ordinal_mapping = {anchor: i for i, anchor in enumerate(anchor_sums.index)}
product_ordinal_mapping = {product: i for i, product in enumerate(product_sums.index)}

# Aplica Ordinal encoding
encoder_color = OrdinalEncoder(categories=[list(color_ordinal_mapping.keys())])
encoder_sku = OrdinalEncoder(categories=[list(sku_ordinal_mapping.keys())])
encoder_anchor = OrdinalEncoder(categories=[list(anchor_ordinal_mapping.keys())])
encoder_product = OrdinalEncoder(categories=[list(product_ordinal_mapping.keys())])

# Cria as novas colunas
df_fixed_encoded['color_encoded'] = encoder_color.fit_transform(df_fixed_encoded[['sku_color']])
df_fixed_encoded['sku_encoded'] = encoder_sku.fit_transform(df_fixed_encoded[['sku']])
df_fixed_encoded['anchor_category_encoded'] = encoder_anchor.fit_transform(df_fixed_encoded[['anchor_category']])
df_fixed_encoded['product_category_encoded'] = encoder_product.fit_transform(df_fixed_encoded[['product_category']])

df_encoded = df_fixed_encoded.drop(['sku','anchor_category','product_category','sku_color'], axis = 1)
df_test = df_fixed_encoded.drop(['sku','anchor_category','product_category','sku_color'], axis = 1)
display(df_encoded)

**Tratamento de datas**


---
A coluna "date" possui uma alta relevância para nosso modelo, porém necessitava ser modificada para se adaptar melhor ao funcionamento dele.

Transformamos seu formato de Ano / Mês / Dia para uma divisão em diversas colunas representando. Assim, o modelo consegue ler melhor os dados de data uma vez que eles foram passados de String > DateTime > Números, facilitando o entendimento da máquina e do modelo.


In [ ]:
# Transformar os valores de "String" de data em "DateTime"
df_test['date'] = pd.to_datetime(df_encoded['date'], format='%Y-%m-%d')
df_encoded['date'] = pd.to_datetime(df_encoded['date'], format='%Y-%m-%d')

# Extraindo os componentes da date
df_encoded['ano'] = df_encoded['date'].dt.year
df_encoded['mes'] = df_encoded['date'].dt.month
df_encoded['dia'] = df_encoded['date'].dt.day
df_encoded['dia_da_semana'] = df_encoded['date'].dt.weekday  # 0: Segunda-feira, 1: Terça-feira, etc.
df_encoded['semana_do_ano'] = df_encoded['date'].dt.isocalendar().week
df_encoded['trimestre'] = df_encoded['date'].dt.quarter
df_encoded['dia_do_ano'] = df_encoded['date'].dt.dayofyear
df_encoded['eh_fim_de_semana'] = df_encoded['date'].dt.weekday >= 5  # Retorna True para sábado e domingo

# Remoção das colunas antigas
df_dates_fixed = df_encoded.drop(['date','weekday_name'], axis = 1)
df_test

**Normalização de colunas e outliers**


---
Os dados numéricos foram analisados para verificar a necessidade de tratamento de outliers, valores muito grandes ou destoantes em relação aos outros no dataframe.

Foi realizada uma normalização onde esses valores foram transformados em valores entre -1 e 1, mantendo sua grandeza entre si, mas não influenciando por seu tamanho e sim mais por sua força de correlação com os demais atributos.

In [ ]:
# Seleção de características para serem normalizadas
df_scale = df_test[['avg_website_visits_last_week','sku_height','sku_width','sku_length','sku_weight','unit_price','supplier_delivery_time']]
df_toScale = df_dates_fixed[['avg_website_visits_last_week','sku_height','sku_width','sku_length','sku_weight','unit_price','supplier_delivery_time']]

# Criando o objeto StandardScaler
scaler = StandardScaler()

# Ajustando e transformando os dados
df_scaled = pd.DataFrame(scaler.fit_transform(df_toScale), columns=df_toScale.columns)
df_jaaj =  pd.DataFrame(scaler.fit_transform(df_scale), columns=df_scale.columns)
# Removendo as colunas antigas e unindo as novas
replace = df_dates_fixed.drop(['avg_website_visits_last_week','sku_height','sku_width','sku_length','sku_weight','unit_price','supplier_delivery_time'], axis = 1)
reprace = df_test.drop(['avg_website_visits_last_week','sku_height','sku_width','sku_length','sku_weight','unit_price','supplier_delivery_time'], axis = 1)
df_final = replace.join(df_scaled, rsuffix='_right')
df_ending = replace.join(df_jaaj, rsuffix='_right')
display(df_ending)

In [ ]:
df_final = df_final.drop(df_final[(df_final['semana_do_ano'] >= 46) & (df_final['semana_do_ano'] <= 51)].index)
df_final = df_final.reset_index(drop = True)
df_final

# Hipóteses



As hipóteses são possíveis explicações das tendências comportamentamentais dos dados, ajudando a compreensão deste e guiando enfim a acurácia mais precisa do modelo preditivo.

###Hipótese #1

Como explicado pelos internos da Mobly, o "crossdocking" é um sistema em que os itens são entregues diretamente do fornecedor para os centros de distribuição encaminharem para os clientes, sem passar pelo estoque. Esse método é utilizado quando não há o produto no estoque. Fizemos uma análise das colunas de 'shipment_type', 'sotck_qty' e 'items_sold', notando que algumas informações se destoam das que foram informadas.


É possível visualizar no gráfico abaixo que a maior parte das vendas ocorreram em produtos que já estavam em estoque.


In [ ]:
# Classificando os produtos
produtos_vazios_vendidos = df_final[df_final['stock_qty'] <= 0]
produtos_com_estoque_vendidos = df_final[df_final['stock_qty'] > 0]

#Contando o número de produtos em cada categoria
num_produtos_vazios_vendidos = len(produtos_vazios_vendidos)
num_produtos_com_estoque_vendidos = len(produtos_com_estoque_vendidos)

# Criando o gráfico de pizza
labels = ['Produtos Sem Estoque Vendidos', 'Produtos com Estoque Vendidos']
sizes = [num_produtos_vazios_vendidos, num_produtos_com_estoque_vendidos]
colors = ['#ff9999', '#66b2ff']
explode = (0.1, 0)  # Explode 1st slice

# Plot
plt.figure(figsize=(10, 6))
plt.pie(sizes, explode=explode, labels=labels, colors=colors, autopct='%1.1f%%', shadow=True, startangle=140)
plt.axis('equal')  # Equal aspect ratio ensures that pie is drawn as a circle.
plt.show()

No entanto, neste segundo gráfico, obtemos a informação de que mais de 70% das vendas foram realizadas por crossdocking.


In [ ]:
crossdocking_and_in_stock = df_final.loc[(df_final['shipment_type_crossdocking'] == 1.0) & (df_final['stock_qty'] > 0)]

# Contando a quantidade de SKUs únicos que fazem crossdocking e estão no estoque
unique_skus_crossdocking_and_in_stock = crossdocking_and_in_stock['sku_encoded'].nunique()

# Contando a quantidade total de SKUs únicos
total_unique_skus = df_final['sku_encoded'].nunique()
labels = [' Com Crossdocking e em Estoque', 'Sem Crossdocking e em Estoque']
# Calculando a quantidade de SKUs que não fazem crossdocking e estão no estoque
unique_skus_not_crossdocking_and_in_stock = total_unique_skus - unique_skus_crossdocking_and_in_stock

# Criando um gráfico de pizza
sizes = [unique_skus_crossdocking_and_in_stock, unique_skus_not_crossdocking_and_in_stock]
colors = ['#FF4500', '#FFD833']
explode = (0.1, 0)  # Explode 1st slice
plt.pie(sizes, explode=explode, labels=labels, colors=colors, autopct='%1.1f%%', shadow=True, startangle=140)
plt.axis('equal')  # Equal aspect ratio ensures that pie is drawn as a circle.
plt.show()

Assim, é apresentado uma aparente contradição nos dados: enquanto a primeira análise sugere que a maioria das vendas provém de estoque próprio, a análise subsequente mostra que, na realidade, a maior parte das vendas é feita através do crossdocking.

Neste terceiro gráfico, é possível concluir que realmente houve a venda de produtos, por crossdocking, que possuíam alta disponibilidade em estoque.

In [ ]:
#Define a separação por cor
colors = df_fixed['shipment_type'].map({'crossdocking': 'turquoise', 'próprio': 'coral'})

#Cria o gráfico de dispersão
ax = df_fixed.plot.scatter(x='items_sold', y='stock_qty', c=colors, edgecolors='w', linewidth=0.5)

#Cria as legendas
legend_elements = [Line2D([0], [0], marker='o', color='w', label='Crossdocking', markerfacecolor='turquoise', markersize=5),
                   Line2D([0], [0], marker='o', color='w', label='Próprio', markerfacecolor='coral', markersize=5)]
ax.legend(handles=legend_elements)

#Apresenta o gráfico
plt.show()

Conseguimos elaborar duas hipóteses sobre o motivo dessa contradição:
* Em algumas situações, um produto pode ter sido vendido em quantidade superior ao estoque disponível e diferente de zero, sendo atendido parcialmente pelo estoque e parcialmente pelo crossdocking. Isso pode levar a classificar a venda inteira como “crossdocking”, distorcendo a representação percentual das vendas e levando à percepção de uma prevalência maior do que realmente existe.

In [ ]:
vendas_mistas = df_encoded[['shipment_type_crossdocking','items_sold', 'stock_qty']]
vendas_mistas[(vendas_mistas['shipment_type_crossdocking'] == 1) & (vendas_mistas['items_sold'] > vendas_mistas['stock_qty']) & (vendas_mistas['stock_qty'] > 1)]

* Em outras situações, o processamento da venda via crossdocking ser realizado mesmo havendo estoque disponível pode ser explicado pelo sistema de vendas agendadas. Neste sistema, o produto é enviado diretamente ao cliente no período escolhido por ele, sem passar pelo estoque convencional.

In [ ]:
vendas_agendadas = df_encoded[['shipment_type_crossdocking','items_sold', 'stock_qty']]
vendas_agendadas[(vendas_agendadas['shipment_type_crossdocking'] == 1) & (vendas_agendadas['items_sold'] > 0) & (vendas_agendadas['stock_qty'] > 0)]

### Hipótese #2
A análise dos gráficos mostra uma tendência curiosa no comportamento dos visitantes do site. Em julho, há um pico de visitas, superado apenas por novembro. Contudo, apesar do grande número de visitantes e dos preços médios mais baixos em julho, o volume de vendas é surpreendentemente baixo. Além disso, julho se destaca por ter muitas visitas e uma grande redução de itens em estoque. Isso levanta a questão: Por que em julho, com alto tráfego e preços reduzidos, as vendas são tão baixas em comparação a outros meses? É necessário investigar as causas desse comportamento atípico.

Abaixo é possível analisar a quantidade de vendas e a média de preço dos produtos por mês em um intervalo de 3 anos (2020, 2021, 2022). Nota-se que em julho as vendas e os preços são baixos.


In [ ]:

df_items_sold = df_final[['mes', 'items_sold']]
grouped_items_sold = df_items_sold.groupby('mes').sum()

df_filtered_price = df_dates_fixed[['mes', 'unit_price']]
grouped_data_price = df_filtered_price.groupby('mes').mean()

df_filtered_stock = df_final[['mes', 'stock_qty']]
grouped_data_stock = df_filtered_stock.groupby('mes').mean()

plt.figure(figsize=(12, 6))

line1, = plt.plot(grouped_items_sold.index, grouped_items_sold['items_sold'], marker='o', linestyle='-', color='orange', label='Quantidade de Vendas')
plt.xlabel('Meses')
plt.ylabel('Quantidade Vendas')
plt.title('Preço Médio/Quantidade de Vendas Vs Meses ')
plt.grid(True)

plt.xticks(list(range(1, 13)), list(range(1, 13)))
ax2 = plt.gca().twinx()
line2, = ax2.plot(grouped_data_price.index, grouped_data_price['unit_price'], marker='o', linestyle='-', color='green', label='Preço Médio')
ax2.set_ylabel('Preço Médio')

lines = [line1, line2]
labels = [l.get_label() for l in lines]
plt.legend(lines, labels, loc='upper left')

plt.show()

Abaixo é possível analisar a média da quantidade no estoque e as visitas no site por mês em um intervalo de 3 anos (2020, 2021, 2022). Nota-se que curiosamente as visitas no site estão muito altas, enquanto o estoque, os preços, e as vendas estão em baixa.


In [ ]:
import matplotlib.pyplot as plt

# Filtrando o DataFrame para pegar apenas as colunas relevantes para a primeira análise
df_items_sold = df_final[['mes', 'avg_website_visits_last_week']]
# Agrupando os dados por 'mes' e calcule a soma dos items vendidos
grouped_items_sold = df_items_sold.groupby('mes').sum()

# Filtrando o DataFrame para pegar apenas as colunas relevantes para a segunda análise
df_filtered_price = df_dates_fixed[['mes', 'stock_qty']]
# Agrupando os dados por 'mes' e calcule a média do 'unit_price'
grouped_data_price = df_filtered_price.groupby('mes').mean()

# Filtrando o DataFrame para pegar apenas as colunas relevantes para a terceira análise
df_filtered_stock = df_final[['mes', 'stock_qty']]
# Agrupando os dados por 'mes' e calcule a média do 'stock_qty'
grouped_data_stock = df_filtered_stock.groupby('mes').mean()

# Criando um gráfico de linha juntando todas as informações
plt.figure(figsize=(12, 6))

# Primeiro eixo y (esquerdo)
line1, = plt.plot(grouped_items_sold.index, grouped_items_sold['avg_website_visits_last_week'], marker='o', linestyle='-', color='blue', label='Visitas no Site')
plt.xlabel('Meses')
plt.ylabel('Visitas no Site')
plt.title('Visitas no Site/Média quantidade no Estoque vs Meses')
plt.grid(True)

# Ajustando os ticks do eixo x
plt.xticks(list(range(1, 13)), list(range(1, 13)))

# Criando um segundo eixo y à direita
ax2 = plt.gca().twinx()
line2, = ax2.plot(grouped_data_price.index, grouped_data_price['stock_qty'], marker='o', linestyle='-', color='red', label='Média Quantidade no Estoque')
ax2.set_ylabel('Média Quantidade no Estoque')

# Combinando as legendas
lines = [line1, line2]
labels = [l.get_label() for l in lines]
plt.legend(lines, labels, loc='upper left')

plt.show()


Conseguimos elaborar uma hipótese sobre o motivo dessa contradição:

* Julho é um mês em que muitas empresas fazem liquidações devido à sua posição estratégica no calendário comercial, oferecendo promoções e descontos. Essas ofertas aumentam o tráfego em sites de e-commerce e atraem muitos consumidores. No entanto, surge o desafio da gestão de estoque. Se produtos desejados pelos clientes não estiverem disponíveis, mesmo com preços reduzidos, isso pode afetar negativamente as vendas, frustrando os compradores. Assim, é vital uma boa gestão de estoque durante essas promoções para atender à demanda e garantir a satisfação do cliente.






### Hipótese #3

Os gráficos que relacionam "preço" e "vendas" nos anos de 2020, 2021 e 2022 revelam uma tendência notável. Nos primeiros dois anos, as vendas aumentaram proporcionalmente aos preços, indicando uma relação direta entre eles. No entanto, em 2022, essa tendência mudou, com as vendas diminuindo à medida que os preços aumentavam e vice-versa. Esse novo padrão levanta questionamentos sobre por que as vendas se elevaram em 2020 e 2021, apesar dos aumentos nos preços, enquanto o oposto ocorreu em 2022. Assim, foram propostas hipóteses para explicar essa mudança.

Abaixo é possível analisar a quantidade de vendas e a média de preço dos produtos no ano de 2020. Nota-se que os preços e vendas tendem a seguir mejoritariamente a mesma linha de cresciemento.


In [ ]:
sold_2020 = df_encoded.loc[df_encoded['ano'] == 2020]
df_items_sold = sold_2020[['mes', 'items_sold']]
grouped_items_sold = df_items_sold.groupby('mes').sum()

price_2020 = df_dates_fixed.loc[df_dates_fixed['ano'] == 2020]
df_filtered_price = price_2020[['mes', 'unit_price']]
grouped_data_price = df_filtered_price.groupby('mes').mean()

df_filtered_stock = df_encoded[['mes', 'unit_price']]
grouped_data_stock = df_filtered_stock.groupby('mes').mean()

plt.figure(figsize=(12, 6))
line1, = plt.plot(grouped_items_sold.index, grouped_items_sold['items_sold'], marker='o', linestyle='-', color='blue', label='Média de vendas')
plt.xlabel('Meses')
plt.ylabel('Média de vendas por sku')
plt.title('Preço/Vendas x 2020')
plt.grid(True)

plt.xticks(list(range(1, 13)), list(range(1, 13)))
ax2 = plt.gca().twinx()
line2, = ax2.plot(grouped_data_price.index, grouped_data_price['unit_price'], marker='o', linestyle='-', color='red', label='Média de preço')
ax2.set_ylabel('Preço médio por sku')

lines = [line1, line2]
labels = [l.get_label() for l in lines]

plt.legend(lines, labels, loc='upper left')
plt.show()

Abaixo é possível analisar a quantidade de vendas e a média de preço dos produtos no ano de 2021. Nota-se novamente que os preços e vendas tendem a seguir mejoritariamente a mesma linha de cresciemento.


In [ ]:

sold_2020 = df_encoded.loc[df_encoded['ano'] == 2021]
df_items_sold = sold_2020[['mes', 'items_sold']]
grouped_items_sold = df_items_sold.groupby('mes').sum()

price_2020 = df_dates_fixed.loc[df_dates_fixed['ano'] == 2021]
df_filtered_price = price_2020[['mes', 'unit_price']]
grouped_data_price = df_filtered_price.groupby('mes').mean()

df_filtered_stock = df_encoded[['mes', 'unit_price']]
grouped_data_stock = df_filtered_stock.groupby('mes').mean()

plt.figure(figsize=(12, 6))
line1, = plt.plot(grouped_items_sold.index, grouped_items_sold['items_sold'], marker='o', linestyle='-', color='blue', label='Média de vendas')
plt.xlabel('Meses')
plt.ylabel('Média de vendas por sku')
plt.title('Preço/Vendas x 2021')
plt.grid(True)

plt.xticks(list(range(1, 13)), list(range(1, 13)))
ax2 = plt.gca().twinx()

line2, = ax2.plot(grouped_data_price.index, grouped_data_price['unit_price'], marker='o', linestyle='-', color='red', label='Média de preço')
ax2.set_ylabel('Preço médio por sku')
lines = [line1, line2]
labels = [l.get_label() for l in lines]

plt.legend(lines, labels, loc='upper left')
plt.show()

Abaixo é possível analisar a quantidade de vendas e a média de preço dos produtos no ano de 2022. Diferentemente dos anos anteriores nota-se que os preços e vendas tiveram um crescimento inversamente proporcional, ou seja, enquanto um aumentava o outro diminuia seu valor.


In [ ]:
sold_2020 = df_encoded.loc[df_encoded['ano'] == 2022]
df_items_sold = sold_2020[['mes', 'items_sold']]
grouped_items_sold = df_items_sold.groupby('mes').sum()

price_2020 = df_dates_fixed.loc[df_dates_fixed['ano'] == 2022]
df_filtered_price = price_2020[['mes', 'unit_price']]
grouped_data_price = df_filtered_price.groupby('mes').mean()

df_filtered_stock = df_encoded[['mes', 'unit_price']]
grouped_data_stock = df_filtered_stock.groupby('mes').mean()
plt.figure(figsize=(12, 6))

line1, = plt.plot(grouped_items_sold.index, grouped_items_sold['items_sold'], marker='o', linestyle='-', color='blue', label='Média de vendas')
plt.xlabel('Meses')
plt.ylabel('Média de vendas por sku')
plt.title('Preço/Vendas x 2022')
plt.grid(True)

plt.xticks(list(range(1, 13)), list(range(1, 13)))
ax2 = plt.gca().twinx()

line2, = ax2.plot(grouped_data_price.index, grouped_data_price['unit_price'], marker='o', linestyle='-', color='red', label='Média de preço')
ax2.set_ylabel('Preço médio por sku')
lines = [line1, line2]
labels = [l.get_label() for l in lines]

plt.legend(lines, labels, loc='upper left')
plt.show()

Conseguimos elaborar uma hipótese sobre o motivo dessa quebra no padrão de crescimento:

Fatores Macroeconômicos: Nos anos de 2020 e 2021, a pandemia de COVID-19 causou um aumento significativo no valor do dólar, elevando os preços das matérias-primas e, consequentemente, dos produtos da Mobly. A alta inflação nesse período também impactou os valores. Essas razões explicam o aumento nos preços mostrado nos gráficos de 2020 e 2021.

Impacto da Quarentena: A pandemia impulsionou a adoção do trabalho remoto, levando as pessoas a investirem em móveis para melhorar seus espaços domésticos. Isso resultou em uma demanda crescente por móveis, explicando o aumento nas vendas em 2020 e 2021.

Retomada Econômica e Normalização: Em 2022, a economia brasileira retomou suas atividades, levando à estabilidade dos preços e dos indicadores macroeconômicos. A menor demanda por móveis pós-pandemia levou a vendas estáveis. As vendas parecem ter sido sensíveis aos preços, aumentando quando os preços eram acessíveis e vice-versa. Isso pode explicar a quebra do padrão observado anteriormente.

#Gráficos


###Gráfico de relação das vendas mensais nos anos de 2020 a 2022

In [ ]:
df_final2 = df_final[df_final['ano'] != 2023]
monthly_sales = df_final2.groupby(['mes', 'ano'])['items_sold'].sum().reset_index()
fig = px.line(monthly_sales, x='mes', y='items_sold', animation_frame='ano')
fig.layout.updatemenus[0].buttons[0].args[1]['frame']['duration'] = 2000
fig.layout.updatemenus[0].buttons[0].args[1]['transition']['duration'] = 8000
fig.update_layout(title='Total de vendas mensais por ano de 2020 a 2022')
fig.show()


Este gráfico representa o número de itens vendidos por mês, durante os 3 últimos anos (2020, 2021 e 2022).

Uma das características observadas nessa progressão foi aumento de vendas ao longo de três anos, que pode ser explicado por ser o crescimento da economia e do poder de compra dos consumidores, o que aumenta a demanda por produtos, aumento da popularidade da Mobly e a expansão de sua base de clientes. Além disso, existe um impacto muito grande da pandemia sobre as vendas online, que se tornaram mais comuns no cotidiano.

A segunda observação recorre à tendência de estabilização geral do gráfico de vendas ao longo dos anos, que pode ser atribuída à consistência na oferta de produtos e serviços pela Mobly. A empresa pode ter mantido uma estratégia de vendas consistente ao longo de três anos, o que resultou em flutuações suaves nas vendas.

Por último, foi analisado um pico de vendas em novembro pode ser explicado pela proximidade das festas de fim de ano e a Black Friday, principalmente. Durante esse período, muitas pessoas começam a comprar presentes para amigos e familiares, o que aumenta a demanda por produtos e é um período em que empresas realizam promoções e descontos durante esse período para atrair mais clientes, o que pode contribuir para o aumento das vendas.


###Gráfico de soma das visitas médias ao site por dia da semana

In [ ]:
monthly_sales = df_fixed.groupby(['weekday_name'])['avg_website_visits_last_week'].sum().reset_index()
dias_da_semana = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
monthly_sales['weekday_name'] = pd.Categorical(monthly_sales['weekday_name'], categories=dias_da_semana, ordered=True)
monthly_sales = monthly_sales.sort_values('weekday_name')
fig = px.bar(monthly_sales, x='weekday_name', y='avg_website_visits_last_week')
fig.update_layout(title='Soma das visitas médias ao site por dia da semana')
fig.update_yaxes(range=[12780000, 12880000])
fig.update_traces(marker_color='#FF9345')
fig.show()

A representação ilustra a distribuição da soma da média de visitantes no site durante os dias da semana.

Percebe-se que existem três tendências de visitas no site, uma em relação à baixa durante a terça-feira, outra sobre a estabilidade no final de semana e a última sobre a alta na segunda-feira.

A tendência de baixa nas terças-feiras pode ser explicada porque durante os dias úteis, as pessoas tendem a estar mais ocupadas com o trabalho e outras responsabilidades, o que pode diminuir o tempo disponível para navegar em sites de compras.

Durante o fim de semana, as pessoas geralmente têm mais tempo livre para se dedicar a atividades de lazer, incluindo navegar em sites de compras.

Já o aumento de visitas nas segundas-feiras pode ser explicado pelo fenômeno do ‘Efeito Segunda-feira’. Esse efeito se refere à tendência das pessoas de retornar ao trabalho após o fim de semana com uma atitude mais positiva e produtiva. Isso pode levar a um aumento na busca por atualizações e novidades, incluindo visitas a sites de compras.


### Gráfico de dispersão de preço unitário vs itens vendidos, separados por categoria de produto

In [ ]:
fig = px.scatter(df_fixed, x='items_sold', y='unit_price', color='product_category')
fig.update_layout(
    title='Gráfico de dispersão de preço unitário vs itens vendidos',
    xaxis_title='Itens vendidos',
    yaxis_title='Preço unitário'
)
fig.update_traces(marker=dict(opacity=0.3))
fig.show()

Versão mais leve do mesmo gráfico:

In [ ]:
color_map = {'Sala de Estar': 'red', 'Quarto': 'blue', 'Sala de Jantar': 'green', 'Escritório': 'orange', 'Cozinha': 'purple', 'Móveis Gamer': 'chocolate', 'Área Externa': 'hotpink', 'Jardim ': 'slategray', 'Móveis Infantis': 'gold', 'Móveis ': 'cyan', 'Quarto Infantil Decorado': 'cadetblue', 'Tapetes': 'olive', 'Colchões': 'coral', 'Lavanderia': 'silver', 'Acessórios para Cama': 'turquoise', 'Espelhos': 'violet'}

colors = df_fixed['product_category'].map(color_map)

ax = df_fixed.plot.scatter(x='items_sold', y='unit_price', edgecolor='w', c=colors, linewidth=0.5)

legend_elements = []
for category, color in color_map.items():
    legend_elements.append(Line2D([0], [0], marker='o', color='w', label=category, markerfacecolor=color, markersize=5))

ax.legend(handles=legend_elements, loc='center left', bbox_to_anchor=(1, 0.5))

plt.show()

A representação procura adequar a relação entre preço unitário dos produtos e a quantidade destes itens vendidos.

Sobre esta dispersão, há 2 observações: a primeira é de que as maiores vendas ocorrem quando o preço unitário é mais baixo e pode ser explicada pela lei da oferta e da demanda. Quando o preço de um produto é reduzido, é natural que a demanda por ele aumente, já que mais pessoas podem ter acesso a ele. Isso pode levar a um aumento nas vendas, já que mais consumidores estão dispostos a comprar o produto a um preço mais baixo.

Já a segunda é a tendência de valorização no preço unitário dos móveis da categoria sala de jantar pode ser atribuída ao fator de salas de jantar serem mais comuns especificamente em casas com maior área, que remetem à um poder aquisitivo mais elevado, justificando esse valor a ser pago por produtos da classe, além disso, existem produtos dessa classe que são importados, o que aumenta seu preço de forma substancial.

In [ ]:

df_final.columns

# Preparação dos dados

### Seleção das features para o modelo de machine learning
---
Uma das etapas cruciais na construção de um modelo de machine learning é a seleção criteriosa das características (features) que servirão como entrada para o modelo. Cada feature foi selecionada com base em sua relevância para o problema em questão, e essa decisão é fundamental para garantir que nosso modelo seja capaz de aprender com eficácia e gerar previsões precisas.

### Divisão do conjunto de treino e de teste
---
Para avaliar a capacidade de generalização do modelo, é necessário dividir o conjunto de dados em dois subconjuntos: treino e teste. O conjunto de treino é utilizado para treinar o modelo, enquanto o conjunto de teste é utilizado para avaliar o desempenho do modelo em dados não vistos anteriormente. Essa divisão é importante para evitar que o modelo memorize os dados de treino e não seja capaz de generalizar para novos dados.

### Escalonamento das features
---
O escalonamento é uma etapa importante, pois garante que as features estejam na mesma escala, o que facilita o processo de aprendizagem do modelo. Para isso, utilizamos o StandardScaler, que padroniza as features removendo a média e escalonando para a variância unitária. Dessa forma, as features são transformadas de modo que sua distribuição tenha uma média igual a 0 e um desvio padrão igual a 1.

# Modelo de _Machine Learning_

# Modelos supervisonados de regressão
---
Os modelos de regressão são utilizados para prever valores contínuos, como por exemplo, o preço de um imóvel ou a quantidade de vendas de um produto. Neste projeto, utilizamos os seguintes modelos de regressão:
* Random Forest Regressor

Random Forest Regressor
---
O Random Forest Regressor é um modelo de aprendizado de máquina que utiliza o método de florestas aleatórias para realizar a regressão. Esse modelo é uma extensão do Random Forest Classifier, que é um modelo de classificação. O Random Forest Regressor é um modelo muito poderoso, que apresenta um bom desempenho na maioria dos problemas de regressão. Além disso, ele é capaz de lidar com dados ausentes e valores discrepantes, o que é uma grande vantagem em relação a outros modelos de regressão.

###Seleção das melhores features para o modelo

In [ ]:
# Todas as features para testagem
# Há a remoção de 'items_sold' e 'items_sold_bundle' pois são tanto o target como um de seus subconjuntos
features = [ 'stock_qty', 'dollar_quotation'
       'shipment_type_crossdocking', 'shipment_type_próprio',
       'product_department_Cama e Banho', 'product_department_Decoração',
       'product_department_Gamer', 'product_department_Infantil',
       'product_department_Keva', 'product_department_Móveis',
       'origin_country_Importado', 'origin_country_Nacional',
       'process_costing_no', 'process_costing_yes',
       'flag_bundle_SKU vendido em conjunto ou sozinho',
       'flag_bundle_SKU vendido somente sozinho', 'color_encoded',
       'sku_encoded', 'anchor_category_encoded', 'product_category_encoded',
       'ano', 'mes', 'dia', 'dia_da_semana', 'semana_do_ano', 'trimestre',
       'dia_do_ano', 'eh_fim_de_semana', 'avg_website_visits_last_week',
       'sku_height', 'sku_width', 'sku_length', 'sku_weight', 'unit_price',
       'supplier_delivery_time']

In [ ]:
#Criando um dataframe a partir do df_final, com todas as colunas como feature
# Pegando uma amostragem de apenas 160 mil linhas
n = 160000
df_parameters_sampled = df_final.drop(df_final.columns.difference(features), 1).sample(n,random_state = 42)

# Separando as variáveis independentes e dependentes (X e Y)
X2 = df_parameters_sampled
y2 = df_final['items_sold'].sample(n,random_state = 42)
# Dividindo o conjunto de dados em conjuntos de treinamento e teste
X2_train, X2_test, y2_train, y2_test = train_test_split(X2, y2, test_size=0.2, random_state=95)

In [ ]:
# Normalizando as features (escalando para média 0 e desvio padrão 1)
scaler = StandardScaler()
X2_train = scaler.fit_transform(X2_train)
X2_test = scaler.transform(X2_test)

In [ ]:
rf = RandomForestRegressor()
scorer = make_scorer(mean_squared_error, greater_is_better=False)
rfe = RFECV(rf, step=1, cv=5, scoring=scorer, n_jobs=-1)
rfe = rfe.fit(X2_train, y2_train)
print("Feature ranking: ", rfe.ranking_)

In [ ]:
mask = rfe.get_support()

# Convertendo a mascara para um array de numpy para indexação
mask = np.array(mask)

# Usando indexação booleana para pegar as features selecionadas
best_features = [feature for i, feature in enumerate(features) if mask[i]]

print("All features: ", X2.shape[1])
print(features)

print("Selected best: ", len(best_features))
print(best_features)

###Seleção dos melhores parâmetros

In [ ]:
# Features escolhidas anteriormente para o modelo
features = ['product_department_Cama e Banho',  'sku_encoded', 'anchor_category_encoded', 'ano', 'dia_da_semana', 'semana_do_ano', 'dia_do_ano', 'avg_website_visits_last_week', 'sku_height', 'sku_width', 'sku_length', 'sku_weight', 'unit_price', ]

In [ ]:
#Criando um dataframe a partir do df_final, mas somente com as colunas das features definidas
# Pegando uma amostragem de apenas 160 mil linhas
n = 160000
df_parameters_sampled = df_final.drop(df_final.columns.difference(features), 1).sample(n,random_state = 42)

# Separando as variáveis independentes e dependentes (X e Y)
X2 = df_parameters_sampled
y2 = df_final['items_sold'].sample(n,random_state = 42)

# Dividindo o conjunto de dados em conjuntos de treinamento e teste
X2_train, X2_test, y2_train, y2_test = train_test_split(X2, y2, test_size=0.2, random_state=95)

In [ ]:
#Criando um dataframe a partir do df_final, agora readaptado para as features selecionadas
df_parameters = df_final.drop(df_final.columns.difference(features), 1)

# Separando as variáveis independentes e dependentes (X e Y)
X = df_parameters
y = df_final['items_sold']
# Dividindo o conjunto de dados em conjuntos de treinamento e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=95)

In [ ]:
# Normalizando as features (escalando para média 0 e desvio padrão 1)
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)
X2_train = scaler.fit_transform(X2_train)
X2_test = scaler.transform(X2_test)

In [ ]:
# Criando a grid de parâmetros a serem testados
param_grid = {
    'n_estimators': [150,200,250],
    'max_depth': [10,15,20],
}

# Definindo o modelo a ser testado
rf_regressor = RandomForestRegressor()

# Definindo o "score" para avaliarmos qual será a melhor combinação de hiperparâmetros para o modelo
scorer = make_scorer(mean_squared_error, greater_is_better=False)

In [ ]:
# Realizando o grid search (remover n_jobs = -1 caso não queira usar todos os cores do computador)
grid_search = GridSearchCV(estimator=rf_regressor, param_grid=param_grid, cv=5,scoring=scorer, n_jobs=-1, refit=True)

grid_search.fit(X2_train, y2_train)

print("Melhores Hiperparâmetros: ", grid_search.best_params_)

# Conseguindo os melhores estimadores
best_rf_model = grid_search.best_estimator_

print('Best Model', best_rf_model)

###Aplicação do modelo com os hiperparâmetros e features

In [ ]:
# Instanciando o modelo com os hiperparâmetros definidos
rf = RandomForestRegressor()
rf.fit(X_train, y_train)

#Gerando as predições
predictions = rf.predict(X_test)

In [ ]:
#Métrica do Erro Quadrático Médio
mse = mean_squared_error(y_test, predictions)
print("Mean Squared Error:", mse)

#Métrica da Raiz Quadrada do Erro Médio
RMSE = np.sqrt(mse)
print('RMSE:', RMSE)

#Métrica do Coeficiente de Determinação R² score
r2 = r2_score(y_test, predictions)
print("R2 score:", r2)

Gradient Boosting Regressor
---
adaasdasdasdasdasasdasdas

In [ ]:
# Todas as features para testagem
# Há a remoção de 'items_sold' e 'items_sold_bundle' pois são tanto o target como um de seus subconjuntos
features = [ 'stock_qty', 'dollar_quotation'
       'shipment_type_crossdocking', 'shipment_type_próprio',
       'product_department_Cama e Banho', 'product_department_Decoração',
       'product_department_Gamer', 'product_department_Infantil',
       'product_department_Keva', 'product_department_Móveis',
       'origin_country_Importado', 'origin_country_Nacional',
       'process_costing_no', 'process_costing_yes',
       'flag_bundle_SKU vendido em conjunto ou sozinho',
       'flag_bundle_SKU vendido somente sozinho', 'color_encoded',
       'sku_encoded', 'anchor_category_encoded', 'product_category_encoded',
       'ano', 'mes', 'dia', 'dia_da_semana', 'semana_do_ano', 'trimestre',
       'dia_do_ano', 'eh_fim_de_semana', 'avg_website_visits_last_week',
       'sku_height', 'sku_width', 'sku_length', 'sku_weight', 'unit_price',
       'supplier_delivery_time']

In [ ]:
#Criando um dataframe a partir do df_final, com todas as colunas como feature
# Pegando uma amostragem de apenas 160 mil linhas
n = 160000
df_parameters_sampled = df_final.drop(df_final.columns.difference(features), 1).sample(n,random_state = 42)

# Separando as variáveis independentes e dependentes (X e Y)
X2 = df_parameters_sampled
y2 = df_final['items_sold'].sample(n,random_state = 42)
# Dividindo o conjunto de dados em conjuntos de treinamento e teste
X2_train, X2_test, y2_train, y2_test = train_test_split(X2, y2, test_size=0.2, random_state=95)

In [ ]:
# Normalizando as features (escalando para média 0 e desvio padrão 1)
scaler = StandardScaler()
X2_train = scaler.fit_transform(X2_train)
X2_test = scaler.transform(X2_test)

In [ ]:
clf = GradientBoostingRegressor()
scorer = make_scorer(mean_squared_error, greater_is_better=False)
rfe = RFECV(clf, step=1, cv=5, scoring=scorer, n_jobs=-1)
rfe = rfe.fit(X2_train, y2_train)
print("Feature ranking: ", rfe.ranking_)

In [ ]:
mask = rfe.get_support()

# Convertendo a mascara para um array de numpy para indexação
mask = np.array(mask)

# Usando indexação booleana para pegar as features selecionadas
best_features = [feature for i, feature in enumerate(features) if mask[i]]

print("All features: ", X2.shape[1])
print(features)

print("Selected best: ", len(best_features))
print(best_features)

In [ ]:
# Features escolhidas anteriormente para o modelo
features = ['product_department_Cama e Banho',  'sku_encoded', 'anchor_category_encoded', 'ano', 'dia_da_semana', 'semana_do_ano', 'dia_do_ano', 'avg_website_visits_last_week', 'sku_height', 'sku_width', 'sku_length', 'sku_weight', 'unit_price', ]

In [ ]:
#Criando um dataframe a partir do df_final, mas somente com as colunas das features definidas
# Pegando uma amostragem de apenas 160 mil linhas
n = 160000
df_parameters_sampled = df_final.drop(df_final.columns.difference(features), 1).sample(n,random_state = 42)

# Separando as variáveis independentes e dependentes (X e Y)
X2 = df_parameters_sampled
y2 = df_final['items_sold'].sample(n,random_state = 42)
# Dividindo o conjunto de dados em conjuntos de treinamento e teste
X2_train, X2_test, y2_train, y2_test = train_test_split(X2, y2, test_size=0.2, random_state=95)

In [ ]:
#Criando um dataframe a partir do df_final, agora readaptado para as features selecionadas
df_parameters = df_final.drop(df_final.columns.difference(features), 1)

# Separando as variáveis independentes e dependentes (X e Y)
X = df_parameters
y = df_final['items_sold']
# Dividindo o conjunto de dados em conjuntos de treinamento e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=95)

In [ ]:
# Normalizando as features (escalando para média 0 e desvio padrão 1)
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)
X2_train = scaler.fit_transform(X2_train)
X2_test = scaler.transform(X2_test)

In [ ]:
# Criando a grid de parâmetros a serem testados
param_grid = {
    'n_estimators': [230,250,270],
    'max_depth': [8,10,12],
    'learning_rate': [00.07,0.08,0.09]
}

# Definindo o modelo a ser testado
gb_regressor = GradientBoostingRegressor()

# Definindo o "score" para avaliarmos qual será a melhor combinação de hiperparâmetros para o modelo
scorer = make_scorer(mean_squared_error, greater_is_better=False)

In [ ]:
# Realizando o grid search (remover n_jobs = -1 caso não queira usar todos os cores do computador)
grid_search = GridSearchCV(estimator=gb_regressor, param_grid=param_grid, cv=5,scoring=scorer, n_jobs=-1, refit=True)

grid_search.fit(X2_train, y2_train)

print("Melhores Hiperparâmetros: ", grid_search.best_params_)

# Conseguindo os melhores estimadores
best_gb_model = grid_search.best_estimator_

print('Best Model', best_gb_model)

In [ ]:
# Instanciando o modelo com os hiperparâmetros definidos
gbr = GradientBoostingRegressor(n_estimators=270,max_depth=10,learning_rate=0.09)
gbr.fit(X_train, y_train)

#Gerando as predições
predictions = gbr.predict(X_test)

#Métrica do Erro Quadrático Médio
mse = mean_squared_error(y_test, predictions)
print("Mean Squared Error:", mse)

#Métrica da Raiz Quadrada do Erro Médio
RMSE = np.sqrt(mse)
print('RMSE:', RMSE)

#Métrica do Coeficiente de Determinação R² score
r2 = r2_score(y_test, predictions)
print("R2 score:", r2)

Extreme Gradient Boosting Regressor(XGBoost)
---
O XGBoost (Extreme Gradient Boosting) é um algoritmo de aprendizado de máquina usado para problemas de regressão. Ele funciona construindo uma série de árvores de decisão, cada uma corrigindo os erros da árvore anterior. As árvores são ponderadas e combinadas para prever valores contínuos, tornando-o eficaz para modelar relações complexas entre variáveis e prever resultados numéricos com precisão.

### Seleção das melhores features para o modelo

In [ ]:
# Todas as features para testagem
# Há a remoção de 'items_sold' e 'items_sold_bundle' pois são tanto o target como um de seus subconjuntos
features = [ 'stock_qty', 'dollar_quotation'
       'shipment_type_crossdocking', 'shipment_type_próprio',
       'product_department_Cama e Banho', 'product_department_Decoração',
       'product_department_Gamer', 'product_department_Infantil',
       'product_department_Keva', 'product_department_Móveis',
       'origin_country_Importado', 'origin_country_Nacional',
       'process_costing_no', 'process_costing_yes',
       'flag_bundle_SKU vendido em conjunto ou sozinho',
       'flag_bundle_SKU vendido somente sozinho', 'color_encoded',
       'sku_encoded', 'anchor_category_encoded', 'product_category_encoded',
       'ano', 'mes', 'dia', 'dia_da_semana', 'semana_do_ano', 'trimestre',
       'dia_do_ano', 'eh_fim_de_semana', 'avg_website_visits_last_week',
       'sku_height', 'sku_width', 'sku_length', 'sku_weight', 'unit_price',
       'supplier_delivery_time']

In [ ]:
#Criando um dataframe a partir do df_final, com todas as colunas como feature
# Pegando uma amostragem de apenas 160 mil linhas
n = 160000
df_parameters_sampled = df_final.drop(df_final.columns.difference(features), 1).sample(n,random_state = 42)

# Separando as variáveis independentes e dependentes (X e Y)
X2 = df_parameters_sampled
y2 = df_final['items_sold'].sample(n,random_state = 42)
# Dividindo o conjunto de dados em conjuntos de treinamento e teste
X2_train, X2_test, y2_train, y2_test = train_test_split(X2, y2, test_size=0.2, random_state=95)

In [ ]:
# Normalizando as features (escalando para média 0 e desvio padrão 1)
scaler = StandardScaler()
X2_train = scaler.fit_transform(X2_train)
X2_test = scaler.transform(X2_test)

In [ ]:
from xgboost import XGBRegressor
xgb_reg = XGBRegressor()
scorer = make_scorer(mean_squared_error, greater_is_better=False)
rfe = RFECV(xgb_reg, step=1, cv=5, scoring=scorer)
rfe = rfe.fit(X2_train, y2_train)
print("Feature ranking: ", rfe.ranking_)

In [ ]:
mask = rfe.get_support()

# Convertendo a mascara para um array de numpy para indexação
mask = np.array(mask)

# Usando indexação booleana para pegar as features selecionadas
best_features = [feature for i, feature in enumerate(features) if mask[i]]

print("All features: ", X2.shape[1])
print(features)

print("Selected best: ", len(best_features))
print(best_features)

### Seleção dos melhores parâmetros

In [ ]:
# Features escolhidas anteriormente para o modelo
features = ['shipment_type_crossdocking', 'product_department_Cama e Banho', 'product_department_Gamer', 'poduct_department_Infantil', 'product_department_Móveis', 'origin_country_Importado', 'process_costing_no', 'flag_bundle_SKU vendido em conjunto ou sozinho', 'color_encoded', 'sku_encoded', 'anchor_category_encoded', 'ano', 'mes', 'dia_da_semana', 'semana_do_ano', 'dia_do_ano', 'avg_website_visits_last_week', 'sku_height', 'sku_width', 'sku_length', 'sku_weight', 'unit_price', 'supplier_delivery_time']

In [ ]:
#Criando um dataframe a partir do df_final, mas somente com as colunas das features definidas
# Pegando uma amostragem de apenas 160 mil linhas
n = 160000
df_parameters_sampled = df_final.drop(df_final.columns.difference(features), 1).sample(n,random_state = 42)

# Separando as variáveis independentes e dependentes (X e Y)
X2 = df_parameters_sampled
y2 = df_final['items_sold'].sample(n,random_state = 42)

# Dividindo o conjunto de dados em conjuntos de treinamento e teste
X2_train, X2_test, y2_train, y2_test = train_test_split(X2, y2, test_size=0.2, random_state=95)

In [ ]:
#Criando um dataframe a partir do df_final, agora readaptado para as features selecionadas
df_parameters = df_final.drop(df_final.columns.difference(features), 1)

# Separando as variáveis independentes e dependentes (X e Y)
X = df_parameters
y = df_final['items_sold']
# Dividindo o conjunto de dados em conjuntos de treinamento e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=95)

In [ ]:
# Normalizando as features (escalando para média 0 e desvio padrão 1)
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)
X2_train = scaler.fit_transform(X2_train)
X2_test = scaler.transform(X2_test)

In [ ]:
# Criando a grid de parâmetros a serem testados
param_grid = {
    'n_estimators': [10,150,200,250],
    'max_depth': [10,15,20],
    'learning_rate': [0.07,0.08,0.09]
}

# Definindo o modelo a ser testado
xgb_reg = XGBRegressor()

# Definindo o "score" para avaliarmos qual será a melhor combinação de hiperparâmetros para o modelo
scorer = make_scorer(mean_squared_error, greater_is_better=False)

In [ ]:
# Realizando o grid search (remover n_jobs = -1 caso não queira usar todos os cores do computador)
grid_search = GridSearchCV(estimator=xgb_reg, param_grid=param_grid, cv=5,scoring=scorer, refit=True)

grid_search.fit(X2_train, y2_train)

print("Melhores Hiperparâmetros: ", grid_search.best_params_)

# Conseguindo os melhores estimadores
best_xgb_model = grid_search.best_estimator_

print('Best Model', best_xgb_model)

### Aplicação do modelo com os hiperparâmetros e features

In [ ]:
# Instanciando o modelo com os hiperparâmetros definidos
xgb_reg = XGBRegressor(n_estimators=150, max_depth=10, learning_rate=0.09)
xgb_reg.fit(X_train, y_train)

#Gerando as predições
predictions = xgb_reg.predict(X_test)

In [ ]:
#Métrica do Erro Quadrático Médio
mse = mean_squared_error(y_test, predictions)
print("Mean Squared Error:", mse)

#Métrica da Raiz Quadrada do Erro Médio
RMSE = np.sqrt(mse)
print('RMSE:', RMSE)

#Métrica do Coeficiente de Determinação R² score
r2 = r2_score(y_test, predictions)
print("R2 score:", r2)